### Install AI Core Python SDK

In [ ]:
!pip install ai-core-sdk

In [ ]:
import json
import requests
import time
import yaml
from IPython.display import clear_output
from pprint import pprint

from ai_core_sdk.models import ParameterBinding, Status

### 1. Connect to your AI Core instance

In [ ]:
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url = "<YOUR_AI_API_URL>" + "/v2", # The present SAP AI Core API version is 2
    auth_url=  "<YOUR_url>" + "/oauth/token", # Suffix to add
    client_id = "<YOUR_clientid>",
    client_secret = "<YOUR_clientsecret>"
)

*Returns no output, manually check your values.*

Example of expected input, Replace with your own instance's value.

```
ai_core_client = AICoreV2Client(
    base_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com" + "/v2", 
    auth_url=  "https://tutorial.authentication.us10.hana.ondemand.com" + "/oauth/token", 
    client_id = "ss-2222-aaa-4444-888-8555!bbb|aicore!bbb",
    client_secret = "qwertyuiop"
)

```

### 2. Test connection

In [ ]:
response = ai_core_client.repositories.query()
print(response.count) # Should return integer value else your values in above step are incorrect

### 2. Onboard the Git repository that contains the templates

In [ ]:
# WARNING: Refrain from onboarding again if previously onboarded
#  else you will get AIAPIServerException 409

ai_core_client.repositories.create(
    name = "azure-openai-aicore",
    url = "https://github.com/<YOUR_GITHUB_USERNAME>/azure-openai-aicore-cap-api", # Forked repo
    username = "<YOUR_GITHUB_USERNAME>",
    password = "<YOUR_GITHUB_PERSONAL_ACCESS_TOKEN>" # To Generate check: https://developers.sap.com/tutorials/ai-core-helloworld.html#7ab4912e-2277-496b-84d9-7f1c9ef2365c
)

In [ ]:
# Check onboarded repositories
response = ai_core_client.repositories.query()
#
for repository in response.resources:
    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)


*Expected Output*

```
...
Name: azure-openai-aicore
URL: https://github.com/john/azure-openai-aicore-cap-api
Status: RepositoryStatus.COMPLETED
```

### 3. Register an application

In [ ]:
# WARNING: Run only once

ai_core_client.applications.create(
    repository_url = "https://github.com/<YOUR_GITHUB_USERNAME>/azure-openai-aicore-cap-api",
    path = "01-ai-core-azure-openai-proxy/scenario", # Scan this folder for instruction YAML files
    revision = "HEAD"
)

In [ ]:
# List scenarios scanned by the application created above
response = ai_core_client.scenario.query(resource_group='default')

for scenario in response.resources:
    print(scenario.__dict__)

*Expected Output*

```
...
{'id': 'azure-openai-proxy', 'name': 'Azure OpenAI Proxy', 'description': 'Azure OpenAI  Proxy', 'labels': None, ...)}
```

### 4 Docker Hub (optional)
#### 4.1 Register Docker secret on SAP BTP, AI Core (optional)

In [ ]:
docker_secret = {
  "name": "docker-credentials",
  "data": {
    ".dockerconfigjson": "{\"auths\": {\"https://index.docker.io\": {\"username\": \"<USERNAME>\", \"password\": \"<DOCKER-HUB-PAT-TOKEN>\"}}}"
  }
}


response = ai_core_client.docker_registry_secrets.create(
    name = docker_secret["name"],
    data = docker_secret["data"]
)

print(response.__dict__)

*Expected Output*

```
{'message': 'secret has been created'}
```

#### 4.2 Build and push Docker image (optional)
```
$ cd proxy
$ docker build -t {DOCKER_USERNAME}/azure-openai-proxy .
$ docker push {DOCKER_USERNAME}/azure-openai-proxy
```

THROUGH THE DOCKER CLI.  
See: https://developers.sap.com/tutorials/ai-core-aiapi-clientsdk-workflows.html#f824a41d-efe8-4883-8238-caef4ac5f789

### 5. Create a resource group (Optional)

In [ ]:
# For Free Tier AI Core Serice:
#  you will not be able to create a new resource group.
#  resource group named `default` exists in all systems.
#  cell execution does not impact existing contents
#
# For paid AI Core service: 
#  IF you wish execute this step
#  You are NOT REQUIRED TO RE-EXECUTE or Redo previous completed steps.
#  Ensure that in the steps that follows modify with your resource group name

resource_group_id = "default" # For free tier; `default` exsits in all systems
# resource_group_id = "my-openai-proxy-ns" # For paid account you can create a namespace aka resource group
                                              # Other steps 

response = ai_core_client.resource_groups.create(resource_group_id = resource_group_id)
print(response.__dict__)

### 6. Create configuration to serve the model

In [ ]:
OPENAI_API_BASE = "https://<YOUR_VALUE_FROM_AZURE>.openai.azure.com" # Endpoint, Check out Azure guide if you dont know: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?tabs=command-line&pivots=programming-language-python
OPENAI_API_KEY = "<YOUR_API_KEY>"
DOCKER_NAMESPACE = "kayschmitteckert585" # Edit ONLY If you performed Docker step



# No modification required in below snippet
response = ai_core_client.configuration.create(
    name = "azure-proxy-serve",
    scenario_id = "azure-openai-proxy",
    executable_id = "azure-openai-proxy",
    input_artifact_bindings = [],
    parameter_bindings = [
        ParameterBinding(key = "OPENAI_API_BASE", value = OPENAI_API_BASE),
        ParameterBinding(key = "OPENAI_API_KEY", value = OPENAI_API_KEY), 
        ParameterBinding(key = "DOCKER_NAMESPACE", value = DOCKER_NAMESPACE)
    ],
    resource_group = "default"
)


serve_config_resp = response
print(response.__dict__)

### 7. Actually serve the proxy

In [ ]:
# Start proxy
response = ai_core_client.deployment.create(
    configuration_id=serve_config_resp.id,
    resource_group='default'
)

deployment_resp = response
print(response.__dict__)

In [ ]:
# Poll deployment status.
# No modification required in below snipet
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    # Get Status
    #
    deployment = response = ai_core_client.deployment.get(
        deployment_id=deployment_resp.id,
        resource_group="default"
    )
    status = deployment.status
    print("...... deployment status ......", flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")

# Allow some time for deployment URL to get ready.
time.sleep(10)

### 8. Do an inference request

In [ ]:
endpoint = f"{deployment.deployment_url}/v2/envs"
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "ai-resource-group": "default",
           "Content-Type": "application/json"}
response = requests.get(endpoint, headers=headers)

legacy_davinci = False # set True if you have a davinci model deployment on Azure OpenAI Services
if legacy_davinci:
    body = {
        "engine": "<YOUR ENGINE>", # include your davinci engine from a deployment of an Azure OpenAI services model
        "prompt": "Classify the following news article into 1 of the following categories: categories: [Business, Tech, Politics, Sport, Entertainment]\n\nnews article: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective:\n\nClassified category:",
        "max_tokens": 60,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "top_p": 1,
        "best_of": 1,
        "stop": "null"
    }
    endpoint = f"{deployment.deployment_url}/v2/completion"
else:
    body = {
        "engine": "<YOUR ENGINE>", # include your engine from a deployment of an Azure OpenAI services model
        "prompt": "Classify the following news article into 1 of the following categories: categories: [Business, Tech, Politics, Sport, Entertainment]\n\nnews article: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective:\n\nClassified category:",
        "max_tokens": 60,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stop": "null"
    }
    endpoint = f"{deployment.deployment_url}/v2/chat-completion"

headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "ai-resource-group": "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=body)

print("Inference result:", response.json())
pprint(vars(response))

### 9. Kill deployment (optional)

In [ ]:
delete_resp = ai_core_client.deployment.modify(deployment_resp.id,
                                                 target_status=Status.STOPPED,
                                              resource_group="default")
status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_core_client.deployment.get(deployment_resp.id, resource_group="default")
    status = deployment.status
    print("...... killing deployment ......", flush=True)
    print(f"Deployment status: {deployment.status}")